In [1]:
import pandas as pd
import numpy as np

In [2]:
agri = pd.read_excel('Agri_analysis.xlsx',sheet_name='clean_data', skiprows=[0,1,3],
                     usecols=['Market Center','Arrivals(in tonnes)','Variety','Maximum Prices(in Rs/quintals)'])
agri = agri.rename(columns={'Market Center':'Market','Arrivals(in tonnes)':'Qty (tonnes)',
                     'Variety':'Crop_variety','Maximum Prices(in Rs/quintals)':'Prices (Rs/quintals)'})
agri = agri.dropna(subset=['Market'])
agri.index = pd.RangeIndex(start=0, stop=agri.shape[0], step=1)

agri['Crop']=np.nan
agri['State']=np.nan

ordered_columns = ['Crop','Crop_variety','Market','State','Qty (tonnes)','Prices (Rs/quintals)']
agri = agri.reindex(columns=ordered_columns)

In [3]:
def get_crop_state(df):
    i = 0
    agri = df
    while i in range(agri.shape[0]):
        if pd.isnull(agri.loc[i,'Prices (Rs/quintals)']):
            if pd.isnull(agri.loc[i+1,'Prices (Rs/quintals)']):
                agri.loc[i+2,'Crop'] = agri.loc[i,'Market']
                agri = agri.drop(i)
            else:
                agri.loc[i+1,'State'] = agri.loc[i,'Market']
                agri = agri.drop(i)
        i += 1
        agri.index = pd.RangeIndex(start=1, stop=agri.shape[0]+1, step=1)
    agri.loc[:,['Crop','State']] = agri.loc[:,['Crop','State']].ffill()
    agri['Qty (tonnes)'] = pd.to_numeric(agri['Qty (tonnes)'],errors='coerce')
    agri['Prices (Rs/quintals)'] = pd.to_numeric(agri['Prices (Rs/quintals)'],errors='coerce',downcast='signed')
    agri = agri.dropna(subset=['Crop_variety','Qty (tonnes)','Prices (Rs/quintals)'])
    agri.index = pd.RangeIndex(start=1, stop=agri.shape[0]+1, step=1)
    return agri

In [4]:
agri_neat = get_crop_state(agri)

In [5]:
agri_neat = agri_neat.sort_values(['Crop','Crop_variety','Prices (Rs/quintals)'],ascending=[True,True,False]).reset_index(drop=True)

In [14]:
agri_neat.to_csv('Agri_commodity_analysed.csv')